In [1]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()
#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset5.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[4,5,6,8])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)
ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=9, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                             

Instructions for updating:
non-resource variables are not supported in the long term
Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.151269
                      0.901225
                      0.178708
                      0.349725
                      1.087852
                      0.130086
                      0.556049
                      0.214854
                      0.278514
 DT - feature_importance_vals
                     0.007282
                     0.156020
                     0.016765
                     0.025439
                     0.202034
                     0.016786
                     0.097594
                     0.009561
                     0.036145
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


keras is no longer supported, please use tf.keras instead.


 NN - feature_importance_vals
                     0.022183
                     0.131890
                     0.023160
                     0.034482
                     0.194628
                     0.029616
                     0.053943
                     0.006464
                     0.033372
Iteration 2


...

 XGB - feature_importance_vals
                      0.145216
                      1.018098
                      0.156428
                      0.372660
                      0.921884
                      0.092526
                      0.546345
                      0.208675
                      0.262505
 DT - feature_importance_vals
                     0.005935
                     0.178628
                     0.005908
                     0.027803
                     0.161767
                     0.015428
                     0.094989
                     0.007616
                     0.032744
 NN - feature_importance_vals
                     0.021887
                     0.105718
                     0.019103
                     0.049146
                     0.125663
                     0.025067
                     0.053903
                     0.006226
                     0.033328
Iteration 9


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.137664
                      1.022980
                      0.159897
                      0.353096
                      0.919078
                      0.102529
                      0.586259
                      0.211039
                      0.263606
 DT - feature_importance_vals
                     0.009116
                     0.176240
                     0.013087
                     0.027445
                     0.164807
                     0.016892
                     0.101281
                     0.008387
                     0.025360
 NN - feature_importance_vals
                     0.021235
                     0.109711
                     0.041027
                     0.040871
                     0.114746
                     0.018883
                     0.054579
                     0.008902
                     0.035382
Iteration 10


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.142292
                      1.016085
                      0.146087
                      0.360989
                      0.927955
                      0.115549
                      0.557674
                      0.217722
                      0.280805
 DT - feature_importance_vals
                     0.004525
                     0.176083
                     0.008577
                     0.026449
                     0.156958
                     0.014518
                     0.099481
                     0.011410
                     0.037879
 NN - feature_importance_vals
                     0.022927
                     0.093742
                     0.026270
                     0.062499
                     0.118369
                     0.026640
                     0.059611
                     0.011952
                     0.039515
